In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from db import *
from psycopg2.errors import ForeignKeyViolation, UniqueViolation
from sklearn.pipeline import Pipeline
from sqlalchemy import delete, exc
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import Session
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

/home/jovyan/work/db.py:8: SAWarning: Did not recognize type 'geometry' of column 'geom'
  mapping_hotel = Table(


In [2]:
hotels = pd.read_sql(
    sql=select(
        mapping_hotel.c.id,
        mapping_hotel.c.hotel_id,
        mapping_hotel.c.is_charter,
    ),
    con=engine.connect(),
).set_index("id")

rooms = pd.read_sql(
    sql=select(
        accommodation_hotel_room.c.id,
        accommodation_hotel_room.c.hotel_id,
        accommodation_hotel_room.c.name,
        mapping_hotel_room.c.room_code,
        mapping_hotel_room.c.room_type,
    ).join_from(mapping_hotel_room, accommodation_hotel_room),
    con=engine.connect(),
)

meals = pd.read_sql(
    sql=select(
        definitions_meal_plan.c.id.label("meal_id"),
        definitions_meal_plan.c.code,
    ),
    con=engine.connect(),
).set_index("code")

operators = pd.read_sql(
    sql=select(
        mapping_operator.c.operator_id,
        mapping_operator.c.external_id,
    ),
    con=engine.connect(),
).set_index("external_id")

In [3]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        # ("data_encoder", BookingDataEncoder(hotels, rooms, meals, operators)),
    ]
)

In [4]:
for destination in ["DU"]:
    booking_data = BookingData(
        destination, "11/01/2022 00:00:00", "10/31/2023 00:00:00"
    )
    get_bookings = booking_data.get()

    if get_bookings is not None:
        df = pipeline.fit_transform(get_bookings)

In [5]:
base_dir = Path(".").resolve()

top_hotel = pd.read_csv(base_dir / "data" / "top_hotels_rr.csv")

In [6]:
df = df[df["hotel_id"].isin(top_hotel["HotelID"])]

In [7]:
df = df.merge(top_hotel, left_on="hotel_id", right_on="HotelID", how="left")

In [8]:
# Operator Code
df["operator_code"] = df["operator_code"].fillna(df["hotel_id"]).apply(str)
# Text Case
for col in ["guest_name", "room_code", "meal"]:
    df[col] = df[col].str.upper()

for col in ["status", "status4", "status5"]:
    df[col] = df[col].str.capitalize()

In [9]:
# Cancellation Date
# Convert "1900-01-01" to pd.NA
df["cancellation_date"] = df["cancellation_date"].replace("1900-01-01", pd.NA)

# Fill missing values with "last_modified_date" where "status" is "Can"
df.loc[
    df["cancellation_date"].isna() & (df["status"] == "Can"),
    "cancellation_date",
] = df["last_modified_date"]

# Set non-matching "status" values to pd.NA
df.loc[
    ~df["cancellation_date"].isna() & (df["status"] != "Can"),
    "cancellation_date",
] = pd.NA

In [10]:
# Update Status
df["status"] = df["status"].apply(lambda x: x if x == "Can" else "Ok")

In [11]:
# Price Info
df.loc[df["purchase_price"] < 1, "purchase_price"] = 0
df.loc[df["sales_price"] < 1, "sales_price"] = 0

In [12]:
df = df.apply(lambda x: x.fillna(0) if x.dtype.kind in "biufc" else x)

In [13]:
# Hotel ID, Charter Bool
df = df.merge(
    hotels,
    how="left",
    left_on="hotel_id",
    right_index=True,
    suffixes=("_external", ""),
)
df["hotel_id"] = df["hotel_id"].fillna(-1).astype(int)

In [14]:
df.to_csv("rr_top_hotels.csv", index=False)